# Revista de Informação Contábil – RIC

In [2]:
import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from wordcloud import WordCloud, STOPWORDS
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\igoan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
plt.style.use('seaborn')

In [4]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [5]:
for key_word in key_words_list:
    print(criar_query(key_word))

contabilidade+ambiental
balanço+social


In [19]:
def listar_links_pagina(query, pg_num=1):
    url_search = f"https://periodicos.ufpe.br/revistas/ricontabeis/search/search?query={query}&searchJournal=37&authors=&title=&abstract=&galleyFullText=&suppFiles=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=01&dateFromDay=1&dateFromYear=2012&dateToMonth=12&dateToDay=31&dateToYear=2021&orderBy=&orderDir=&searchPage={pg_num}#results"
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    tbl_results = soup.find('table', {'class': 'listing'})
    a_links = []
    for a in tbl_results.find_all('a'):
           if 'Resumo' in a.text:
                a_links.append(a['href'])
    return a_links

In [28]:
def listar_links(query):
    pg_num = 1
    a_links = [] 
    while True:
        tmp = listar_links_pagina(query, pg_num)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links

In [35]:
def realiza_consulta(key_words_list):
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(query)
    return a_links

In [36]:
key_words_list = ['contabilidade ambiental', 'balanço social']
links_artigos = realiza_consulta(key_words_list)
links_artigos

['https://periodicos.ufpe.br/revistas/ricontabeis/article/view/239961',
 'https://periodicos.ufpe.br/revistas/ricontabeis/article/view/234958',
 'https://periodicos.ufpe.br/revistas/ricontabeis/article/view/240668',
 'https://periodicos.ufpe.br/revistas/ricontabeis/article/view/7948',
 'https://periodicos.ufpe.br/revistas/ricontabeis/article/view/22602',
 'https://periodicos.ufpe.br/revistas/ricontabeis/article/view/7967',
 'https://periodicos.ufpe.br/revistas/ricontabeis/article/view/13371',
 'https://periodicos.ufpe.br/revistas/ricontabeis/article/view/13337',
 'https://periodicos.ufpe.br/revistas/ricontabeis/article/view/13356',
 'https://periodicos.ufpe.br/revistas/ricontabeis/article/view/248711',
 'https://periodicos.ufpe.br/revistas/ricontabeis/article/view/13342',
 'https://periodicos.ufpe.br/revistas/ricontabeis/article/view/7948',
 'https://periodicos.ufpe.br/revistas/ricontabeis/article/view/230529']

In [47]:
link_artigo = links_artigos[0]
page = requests.get(link_artigo)
soup = BeautifulSoup(page.content, 'html.parser')

In [59]:
def captura_titulo(soup):
    title_tag = soup.find('div', {'id': 'articleTitle'})
    title = title_tag.find('h3').text
    return {'título': title}

In [60]:
def captura_ano(soup):
    year_tags = soup.find('div', {'id': 'breadcrumb'})
    year_tag = year_tags.find_all('a')
    year_text = year_tag[1].text
    year = year_text[-5:-1]
    return {'ano': year}

In [61]:
def captura_resumo(soup):
    abstract_tag = soup.find('div', {'id': 'articleAbstract'})
    abstract = abstract_tag.find('div').text
    return {'resumo': abstract}

In [62]:
def captura_palavras_chave(soup):
    kw_tag = soup.find('div', {'id': 'articleSubject'})
    kw = kw_tag.find('div').text.replace(',', '.').replace(';', '.')
    return {'palavras-chave': kw}

In [122]:
def captura_autores(soup):
    authors_tag = soup.find_all('div', {'class': 'authorBio'})
    autores = []
    affiliation_list = []
    for p in authors_tag:
        autor = p.find('em').text
        autores.append(autor)
        affiliation = p.find_all('br')[-2].next_sibling
        affiliation = affiliation.replace('\t', '')
        affiliation_list.append(affiliation)

    affiliation_list = completar_lista(affiliation_list)
    key_affiliation = [f"afilicao{i}" for i in range(1, 7)]
    autores = completar_lista(autores)
    key_autores = [f"autor{i}" for i in range(1, 7)]

    return [dict(zip(key_autores, autores)), dict(zip(key_affiliation, affiliation_list))]

In [119]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [123]:
captura_autores(soup)

[{'autor1': 'Arthur Frederico Lerner',
  'autor2': 'Vanessa Noguez Machado',
  'autor3': 'Romina Batista de Lucena de Souza',
  'autor4': '',
  'autor5': '',
  'autor6': ''},
 {'afilicao1': 'Universidade Federal do Rio Grande do Sul (UFRGS)',
  'afilicao2': 'Universidade Federal do Rio Grande do Sul (UFRGS)',
  'afilicao3': 'Universidade Federal do Rio Grande do Sul (UFRGS)',
  'afilicao4': '',
  'afilicao5': '',
  'afilicao6': ''}]